In [15]:
import pandas as pd
from sklearn import svm, decomposition
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
import os
import random
from sklearn.model_selection import KFold
import pickle
#import seaborn as sns
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
%matplotlib widget
import matplotlib.pyplot as plt
from sklearn import preprocessing
from pandas import Series
import torch
from torch import nn 
from torch.nn import Conv1d, MaxPool1d, Flatten, Linear, LeakyReLU, ReLU, Sigmoid, Softmax, BatchNorm1d, Sequential, AdaptiveAvgPool1d, Dropout
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
#os.environ["CUDA_VISIBLE_DEVICE"] = "1"
torch.cuda.set_device(2)
import time

In [16]:
class CORAL_loss(nn.Module):
	def __init__(self):
		super(CORAL_loss, self).__init__()
		
	def forward(self, source, target):
		d = source.data.shape[1]
		
		# source covariance
		xm = torch.mean(source, 0, keepdim=True) - source
		xc = xm.t() @ xm
		
		# target covariance
		xmt = torch.mean(target, 0, keepdim=True) - target
		xct = xmt.t() @ xmt
		
		# frobenius norm between source and target
		loss = torch.mean(torch.mul((xc - xct), (xc - xct)))
		loss = loss/(4 * d * d)
		
		return loss

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in 
        Focal Loss for Dense Object Detection.

            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])

        The losses are averaged across observations for each minibatch.

        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5), 
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.


    """
    def __init__(self, class_num=2, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha)
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    def forward(self, inputs, targets):
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs, dim=1)

        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)
        #print(class_mask)


        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]

        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()
        #print('probs size= {}'.format(probs.size()))
        #print(probs)

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 
        #print('-----bacth_loss------')
        #print(batch_loss)


        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

In [18]:
class Rsbu(nn.Module):
    def __init__(self):
        super(Rsbu, self).__init__()
        self.model0 = Sequential(
            Conv1d(1, 8, kernel_size=4, padding=1, stride=2),
            LeakyReLU(),
            Conv1d(8, 16, kernel_size=3, padding=1, stride=1),
            LeakyReLU(),
            Conv1d(16, 32, kernel_size=3, padding=1, stride=1)
        )
        self.model1 = Sequential(
            #BatchNorm1d(32),
            LeakyReLU(),
            Conv1d(32, 64, kernel_size=3, padding=1, stride=1),
            #BatchNorm1d(64),
            LeakyReLU(),
            Conv1d(64, 32, kernel_size=3, padding=1, stride=1)
        )     
        
        self.model2 = Sequential(
            #Flatten(),
            Linear(32, 32),
            #BatchNorm1d(32),
            LeakyReLU(),
            Linear(32, 32),
            Sigmoid()     
        )
        
        self.model3 = Sequential(
            #BatchNorm1d(32),
            LeakyReLU(),
            Flatten(),
            Linear(8*32,2),
            LeakyReLU(),
            #Dropout(0.5),               
        )
        
        self.class_layer = Sequential(
            #Linear(16,2),
            Softmax(dim=1)
        )
        self.gap = AdaptiveAvgPool1d(1)
        for m in self.modules():
            if isinstance(m, Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0.1)
            elif isinstance(m, (Linear, BatchNorm1d)):
                nn.init.normal_(m.weight, 0, 0.1)
                nn.init.constant_(m.bias, 0.1)
        
             
    def forward(self, xs, xt):
        xs_bf_thre = self.model0(xs)
        xt_bf_thre = self.model0(xt)
        xs_thre = self.model1(xs_bf_thre)
        xt_thre = self.model1(xt_bf_thre)
        xs_abs = torch.abs(xs_thre)
        xt_abs = torch.abs(xt_thre)
        xs_out = self.gap(xs_abs) 
        xt_out = self.gap(xt_abs)
        xs_out = xs_out.reshape(64, 32)
        xt_out = xt_out.reshape(64, 32)
        xs_a = self.model2(xs_out)
        xt_a = self.model2(xt_out)
        xs_a = xs_a.reshape(64, 32, 1)
        xt_a = xt_a.reshape(64, 32, 1)
        xs_out = xs_out.reshape(64, 32, 1)
        xt_out = xt_out.reshape(64, 32, 1)
        xs_limit = xs_a * xs_out
        xt_limit = xt_a * xt_out
        
        #soft thresholding
        xs_sub = xs_abs - xs_limit
        xt_sub = xt_abs - xt_limit
        xs_zeros = xs_sub - xs_sub
        xt_zeros = xt_sub - xt_sub
        xs_n_sub = torch.max(xs_sub, xs_zeros)
        xt_n_sub = torch.max(xt_sub, xt_zeros)
        xs_soft_thres = xs_n_sub * (torch.sign(xs_thre))
        xt_soft_thres = xt_n_sub * (torch.sign(xt_thre))
        xs_shortcut = xs_soft_thres + xs_bf_thre
        xt_shortcut = xt_soft_thres + xt_bf_thre
        #print(soft_thres.size())
        xs_features = self.model3(xs_shortcut)
        xt_features = self.model3(xt_shortcut)
        
        coral_loss_model = CORAL_loss()
        coral_loss = coral_loss_model(xs_features, xt_features)
        output = self.class_layer(xs_features)
        return output, coral_loss
    
    def predict(self, xt):
        xt_bf_thre = self.model0(xt)
        xt_thre = self.model1(xt_bf_thre)
        xt_abs = torch.abs(xt_thre)
        xt_out = self.gap(xt_abs)
        xt_out = xt_out.reshape(64, 32)
        xt_a = self.model2(xt_out)
        xt_a = xt_a.reshape(64, 32, 1)
        xt_out = xt_out.reshape(64, 32, 1)
        xt_limit = xt_a * xt_out
        
        #soft thresholding
        xt_sub = xt_abs - xt_limit
        xt_zeros = xt_sub - xt_sub
        xt_n_sub = torch.max(xt_sub, xt_zeros)
        xt_soft_thres = xt_n_sub * (torch.sign(xt_thre))
        xt_shortcut = xt_soft_thres + xt_bf_thre
        #print(soft_thres.size())
        
        
        xt_features = self.model3(xt_shortcut)
        output = self.class_layer(xt_features)
        return output



In [19]:
#获取数据所在路径
def obtain_csv(dir_name, purpose):
    data_path = []
    dir_path = os.path.join(dir_name, purpose)
    file_lists = os.listdir(dir_path)
    file_lists.sort()
    if purpose == 'train':
        for file_list in file_lists:
            file_path = os.path.join(dir_path, file_list)
            data_path.append(file_path)
    else:
        file_list = 'data_test.csv'
        data_path = os.path.join(dir_path, file_list)
    return data_path

In [20]:
#获取经过pca处理后的数据与标签
def load_data_KX09(data_path):
    data = pd.read_csv(data_path,encoding='gbk')
    data = data.drop(['卫星时间'], axis=1)
    data = data.dropna(axis=1)
    columns = data.columns
    drop_list = []
    for f in list(columns):
        try:
            data[f] = data[f].astype(float)
        except:
            pass
        if len(list(pd.unique(data[f]))) == 1:
            drop_list.append(f)
    data = data.drop(drop_list, axis=1)
    data = np.array(data)
    y = data[:,-1].astype(int)
    for i in range(len(y)):
        if y[i] == 1:
            y[i] = 0
        else:
            y[i] = 1
    data = np.delete(data, -1, axis=1)
    #归一化+标准化
    scalar = preprocessing.MinMaxScaler()
    data = scalar.fit_transform(data)
    data = preprocessing.scale(data)
    #pca
    n_components = 16
    pca = decomposition.PCA(n_components=16)
    pca_data = pca.fit_transform(data)
    #pca后的归一化
    pca_data = scalar.fit_transform(pca_data)
    #pca_data = preprocessing.scale(pca_data)
    data_label = np.concatenate((pca_data, y[:,None]), axis=1)
    data_size = len(data_label)
    return data_label, data_size

In [21]:
#获取量子卫星0087数据
def load_data_0087(data_path):
    data = pd.read_csv(data_path, encoding='gbk')
    data = data.drop(index=0)
    data = data.dropna(axis=1)
    y = data['errtype3'].astype(int)
    y = np.array(y)
    data = data.drop(['TMKZ001', 'TMKZ002','errtype1', 'errtype2', 'errtype3', 
                      'errtype4', 'errtype5','errtype6', 'errtype7'], axis=1)

    # 去掉只有一个值的列
    columns = data.columns
    drop_list = []
    for f in list(columns):
        try:
            data[f] = data[f].astype(float)
        except:
            pass
        if len(list(pd.unique(data[f]))) == 1:
            drop_list.append(f)
    data = data.drop(drop_list, axis=1)

    # 转onehot
    object_list = []
    for f in list(data.columns):
        if data[f].dtype == 'object':
            object_list.append(f)
    for f in object_list:
        data_dummies = pd.get_dummies(data[f])
        data = data.drop([f], axis=1)
        data = pd.concat([data, data_dummies], axis=1)  
    data = np.array(data)
    
    #归一化+标准化
    scalar = preprocessing.MinMaxScaler()
    data = scalar.fit_transform(data)
    data = preprocessing.scale(data)
    #pca
    n_components = 16
    pca = decomposition.PCA(n_components=16)
    pca_data = pca.fit_transform(data)
    #pca后的归一化
    pca_data = scalar.fit_transform(pca_data)
    #pca_data = preprocessing.scale(pca_data)
    data_label = np.concatenate((pca_data, y[:,None]), axis=1)
    data_size = len(data_label)
    return data_label, data_size

In [22]:
def processing_data(data):
    label = data[:,-1].to(torch.int)
    data = np.delete(data, -1, axis=1)
    data = data.view((-1, 1, 16))
    data = data.to(torch.float32)
    label = label.to(torch.long)
    device = torch.device('cuda:2')
    data = data.to(device)
    label = label.to(device)
    return data, label

In [ ]:
def t_SNE(data):
    X_tsne = TSNE(n_components=2, random_state=10, learning_rate='auto', init='random').fit_transform(data)
    data_tsne = np.concatenate((X_tsne, label[:,None]), axis=1)
    data_tsne = pd.DataFrame(data_tsne)
    return data_tsne

In [ ]:
data_tsne_s.columns = ['dim1_s', 'dim2_s', 'label_s']
data_tsne_t.columns = ['dim1_t', 'dim2_t', 'label_t']
group_s = data_tsne_s.groupby('label_s')
group_t = data_tsne_t.groupby('label_t')
fig = plt.figure(figsize=(5,5), facecolor='white')
for key, value in group_s:
    d = value
    #plt.style.use("classic")
    if key == 0:
        plt.plot(d['dim1_s'], d['dim2_s'], linestyle='',color='k', marker='.', label='A1 anomaly')
        plt.legend(loc='upper right', prop=font)
    else:
        plt.plot(d['dim1_s'], d['dim2_s'], linestyle='',color='k', marker='3', label='B1 anomaly')
        plt.legend(loc='upper left', prop=font)
        
for key, value in group_t:
    d = value
    #plt.style.use("classic")
    if key == 0:
        plt.plot(d['dim1_t'], d['dim2_t'], linestyle='',color='k', marker='x', label='A1 normal', markersize=4)
        plt.legend(loc='upper right', prop=font)
    else:
        plt.plot(d['dim1_t'], d['dim2_t'], linestyle='',color='k', marker='d', label='B1 normal', markersize=4)
        plt.legend(loc='upper left', prop=font)